In [3]:
import pickle
with open('DIM-119.pkl', 'rb') as f:
    database = pickle.load(f)
lidar_data = database['lidar_data']
path = database['path']
print(len(lidar_data))

6


In [2]:
import rerun as rr
import numpy as np

rr.init("test_distort", spawn=True)
def Axes(l=1.0):
    return rr.Arrows3D(
        vectors=[[l, 0, 0], [0, l, 0], [0, 0, l]],
        colors=[[255,0,0], [0,255,0], [0,0,255]],
    )

def visualize_sim_lidar(groups, path):
    points = []
    positions = []
    rotations = []
    
    for i in range(len(groups)):
        transform = np.array(groups[i]['transformStart']).reshape(4,4).T
        position = transform[:3,3]
        rotation = transform[:3,:3]
        points.append(groups[i]['xyz'])
        positions.append(position)
        rotations.append(rotation)
    
    rr.set_time_seconds('sim_time', 0)
    rr.log('/world', rr.Clear(recursive=True))
    rr.log('/world', Axes(0.5), static=True)
    trajectory = path[:, 1:4]
    rr.log('/world/trajectory', rr.LineStrips3D(trajectory))

    for i in range(len(groups)):
        rr.log(f'/world/measurements/{i}', rr.Transform3D(
            translation=positions[i], mat3x3=rotations[i]))
        rr.log(f'/world/measurements/{i}', Axes(1.0))
        rr.log(f'/world/measurements/{i}/points', rr.Points3D(points[i]))

visualize_sim_lidar(lidar_data, path)

[2024-09-24T03:21:23Z INFO  re_sdk_comms::server] Hosting a SDK server over TCP at 0.0.0.0:9876. Connect with the Rerun logging SDK.
[2024-09-24T03:21:23Z INFO  winit::platform_impl::platform::x11::window] Guessed window scale factor: 1
[2024-09-24T03:21:23Z INFO  re_sdk_comms::server] New SDK client connected from: 127.0.0.1:60758
[2024-09-24T03:21:23Z WARN  wgpu_hal::gles::egl] No config found!
[2024-09-24T03:21:23Z WARN  wgpu_hal::gles::egl] EGL says it can present to the window but not natively
[2024-09-24T03:21:23Z INFO  egui_wgpu] There were 3 available wgpu adapters: {backend: Vulkan, device_type: Cpu, name: "llvmpipe (LLVM 12.0.0, 256 bits)", driver: "llvmpipe", driver_info: "Mesa 21.2.6 (LLVM 12.0.0)", vendor: 0x10005}, {backend: Vulkan, device_type: DiscreteGpu, name: "NVIDIA GeForce RTX 4060", driver: "NVIDIA", driver_info: "555.42.02", vendor: 0x10DE, device: 0x2882}, {backend: Gl, device_type: Other, name: "NVIDIA GeForce RTX 4060/PCIe/SSE2", driver: "OpenGL", driver_info: